# Milestone 3

First we have to encode the file so that the SentenceTransformer module can work with it.

In [17]:
import json

json_file_path = "../../milestone_1/datasets/06_University_documents.json"
new_json_file_path = "../datasets/07_University_documents.json"

json_file = open(json_file_path)
data = json.load(json_file)
json_file.close()

with open(new_json_file_path, 'w', encoding="utf-8") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

In [18]:
import json
from sentence_transformers import SentenceTransformer

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_embedding(text):
    # The model.encode() method already returns a list of floats
    return model.encode(text, convert_to_tensor=False).tolist()

json_file_path = "../datasets/07_University_documents.json"
new_json_file_path = "../datasets/08_Vectorized_university_documents.json"

json_file = open(json_file_path, encoding="utf-8")
data = json.load(json_file)
json_file.close()

# Update each document in the JSON data

combined_text=""

for document in data:
    try:
        # Extract fields if they exist, otherwise default to empty strings
        city_wikipedia_text = document.get("city_wikipedia_text", "")
        wikipedia_text = document.get("wikipedia_text", "")

        combined_text = city_wikipedia_text + " " + wikipedia_text 
        
        document["vector"] = get_embedding(combined_text)
    except:
        print("ECXCEPTIOON: ", combined_text[87196])
        
        
with open(new_json_file_path, 'w', encoding="utf-8") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)